In [1]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses , util
import pandas as pd 
import torch

In [2]:

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

In [3]:
df = pd.read_pickle('data/all_data.pkl')
all_pairs = list(df.itertuples(index=False, name=None))
n_examples = len(all_pairs)

In [4]:
all_pairs

[('Frontend developer (Moments)',
  "Infobip Продуктовая компания из Хорватии, разработчик CPaaS- платформы для A2P и облачных коммуникаций. В 2020 нас оценили в 1 млрд долларов, в 2021 мы выросли до 3000 сотрудников, в 2023 планируем провести IPO.  Платформа обрабатывает 30 терабайт новых данных в день и пропускает 40 000 сообщений в секунду. Среди получателей можете оказаться и вы, когда вызываете Uber, запрашиваете детализацию по счёту в Raiffeisen Bank или оформляете подписку на IVI. Мы не просто &quot;рассылаем SMS-ки&quot;, мы помогаем компаниям строить диалог с пользователями через мессенджеры, пуш-уведомления, голосовые и e-mail сообщения. Для этого у нас есть связка SaaS-продуктов на стыке маркетинга, телекома и облачных коммуникаций.  Moments В Санкт-Петербурге мы разрабатываем Moments, новый SaaS-продукт для таргетированного взаимодействия в паре бизнес-клиент. Омниканальность и привязка к пользовательским профилям — главные особенности продукта, они позволяют компаниям взаи

In [5]:
train_examples = []
#MultipleNegativesRankingLoss
for i in range(n_examples):
  example = all_pairs[i]
  # query_embedding = model.encode(example[0], convert_to_tensor=True).to(model.device)
  # query_embedding2 = model.encode(example[1], convert_to_tensor=True).to(model.device)
  # cos_scores = util.cos_sim(query_embedding, query_embedding2 )[0]
  train_examples.append(InputExample(texts=[example[1], example[0]] ))

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [7]:
from sentence_transformers import losses

train_loss = losses.MultipleNegativesRankingLoss(model=model)

In [8]:
num_epochs = 5

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1)

In [9]:
import gc


torch.cuda.empty_cache()
gc.collect()

0

In [10]:
!nvidia-smi

Thu Apr  4 09:56:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.01                 Driver Version: 546.01       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  | 00000000:05:00.0  On |                  N/A |
|  0%   45C    P8              23W / 170W |   1726MiB / 12288MiB |     21%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps) 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2732 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2732 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2732 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2732 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2732 [00:00<?, ?it/s]

In [12]:
!huggingface-cli login

^C


In [13]:
model.save_to_hub(
    "paraphrase-multilingual-mpnet-base-v2_hh_MNRL", 
    exist_ok=True, 
    )

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

'https://huggingface.co/assskelad/paraphrase-multilingual-mpnet-base-v2_hh_MNRL/commit/a5dd40b95b9bcff268da36ed7e771746f6a9a184'

In [25]:
df = pd.read_pickle('data/all_data.pkl')
model = SentenceTransformer('assskelad/paraphrase-multilingual-mpnet-base-v2_hh_MNRL')
names = list(df['Name'])
names_embeddings = model.encode(names,  normalize_embeddings= True , convert_to_tensor=True)

In [29]:
query_embedding = model.encode('разработчик  сайтов  на  java и работа с людьми    без опыта  работы ', convert_to_tensor=True , normalize_embeddings=True).to(model.device)
cos_scores = util.cos_sim( query_embedding , names_embeddings)[0]
top_results = torch.topk(cos_scores, k=1)

In [30]:
top_results

torch.return_types.topk(
values=tensor([0.7124], device='cuda:0'),
indices=tensor([7703], device='cuda:0'))

In [31]:
indices = [i.item() for i in top_results.indices]
for res in df['Name'][indices]:
    print(res)

Data Scientist


In [32]:
df['Description'][7703]

"Обязанности:  Генерация гипотез по улучшению ранжирования контента и участие в реализации алгоритма подбора контента для юзеров; Создание моделей для модерации комментариев; Разработка моделей для рекомендательных систем; Внедрение ML-решений персонализации продуктовых предложений для клиентов (Next best offer); Участие в развитии DS инфраструктуры и внутренних библиотек; Разработка моделей ценообразования;  Что нужно, чтобы присоединиться к нам:  Опыт построения рекомендательных систем; Знания методов машинного обучения и нейронных сетей; Опыт и интерес к исследовательской работе; Опыт работы с TensorFlow/ PyTorch (мы используем PyTorch); Знание алгоритмов и структур данных; Опыт программирования на Python (библиотеки NumPy, Sklearn); Высшее образование: техническое, математическое или по специальностям компьютерная лингвистика, машинное обучение и анализ данных; Поиск аномалий на оборудовании, приоритезация инцидентов; Антифрод и кибербезопасность; Приоритезация строительства/модерн